# Implementing a Voting System

## Background Theory
Here I will summarize the formalism used/developed in Wayland2019. The construction of a generalized voting system, consisting of a set of voters, a set of candidates and a voting rule to compute the outcome of an election is quite standard in the field of social choice theory.
* Let $V$ be a nonempty set of n voters $\{1,...,n\}$
* Let $C$ be the set of m candidates $\{c_1,...,c_m\}$
* In this situation, we assume that each voter is ranking each of the candidates on their ballot to produce a linear order.
* Let $\mathcal{L}(C)$ denote the set of all linear orders on $C$. It follows then that the collection of ballots, which will be referred to as a profile $\mathbb{P}$, is a subset of $\mathcal{L}(C)$. 
* For $i\in V$ let $P_i\in \mathcal{L}(C)$ denote the truthful preferences of the $i^{th}$ voter.
* Then a voting rule, $f$ is nothing more than a function $f:\mathcal{L}(C)\to C$ that picks out a single winner.

### Voting Rules
There are a number of different ways to determine the outcome of an election. This investigation will only consider positional scoring rules. These assign a numerical score to each candidate based on their rank in each $P_i$; the candidate with the maximum score is selected as the winner. 
* Define a scoring vector to be $s = \langle s_1,s_2,...,s_m \rangle$ where for $j=1,...,m-1, s_j\geq s_{j+1}$.
* For $x\in C$, let the function $score(P_i,x) = s_r$. This picks out the appropriate score in the scoring vector assocaited with the $r^{th}$ element in the linear order $P_i$.
* Thus in collecting the total score for $x\in C$ we are computing $total(\mathbb{P},x) = \sum_{i=1}^n score(P_i,x)$
* This allows for a concrete definition of a voting rule $f$ on a given profile $\mathbb{P}$ and scoring vector $s$:
    $$ f(\mathbb{P},s) = \max_{x\in C} total(\mathbb{P},x)$$
##### This formalism allows you to fully generate a postional scoring rule given it's scoring vector. Here are some specifications of a few common voting rules:
* Plurality: the rule given by $\langle 1,...,0 \rangle$.
* k-approval: the rule given by $\langle 1,1,...,0 \rangle$ where you have $k$ 1's followed by zeros.
* Borda: the rule given by $\langle m-1,m-2,...,0 \rangle$

## Implementation

In [14]:
# Imports
library(dplyr)
library(MVN)
library(Hmisc)
library(ggplot2)
library(tidyverse)
devtools::install_github('MEDSL/elections')
library(elections)
library(data.table)

Skipping install of 'elections' from a github remote, the SHA1 (53297d9f) has not changed since last install.
  Use `force = TRUE` to force installation



In [2]:
# Setup an example Election
C <- c("Donald Trump","Joe Biden","Kanye West")
num_voters <- 30
voting_rule <- "Plurality"


In [15]:
# Generate random ballots for the voters to construct a voting profile 

construct_profile <- function(num_voters,candidates) {    
    #' Return n random permutations on the set of canidates as a matrix
    profile <- matrix(data = seq(num_voters*length(candidates)),nrow = num_voters, ncol = length(candidates))
    for (i in seq(num_voters)) {
        ballot <- sample(C, length(C), replace=FALSE)
        profile[i,] <- ballot
    }
    return(profile)
}



#Create a scoring vector for a particular profile and voting method

generate_s_vector <- function(profile,voting_rule) {
    
    m <- dim(profile)[2]
    
    if (voting_rule == "Plurality"){
        s_vector <- replicate(m,0)
        s_vector[1] <- 1
    }
    else if (voting_rule == "Borda"){
        s_vector <- rev(0:(m-1))
    }
    else {
        print("No functionality for this type of voting method yet")
        return
    }
    return(s_vector)
}

In [4]:
# Compute the total score for each candidate
total <- function(profile, s_vector) {
    
    dim <- dim(profile)
    
    
    if (!is.vector(s_vector)) {
        print("Scoring Vector is not a vector")
        return 
    }
    
    if (dim[2] != length(s_vector)) {
        print("Dimensions of score vector and profile do not match")
        return 
    }
    
    df <- as.data.frame(profile)
    colnames(df) <- s_vector
    freq <- (gather(df) %>% group_by(key, value) %>% tally)
    freq$score <- as.numeric(freq$key)*freq$n

    totals <- matrix(nrow = dim[2],ncol = 2)
    for (i in 1:dim(P)[2]) {
            totals[i,] <- c(unique(freq$value[seq(i, dim(freq)[1], dim[2])]),sum(freq$score[seq(i, dim(freq)[1], dim[2])]))
        }
    return(totals)
}

# Determine the winner of the election

f <- function(profile,voting_rule) {
   
    s_vector <- generate_s_vector(profile,voting_rule)
    totals <- as.data.frame(total(profile,s_vector))
    colnames(totals) <- c("Candidate",paste(voting_rule,"Score"))
    print(totals)
    winner <- totals[which.max(totals[,2]),]$Candidate
    print(paste("The winner of the election is",winner))
    return(winner)
    
}

In [21]:
# Test with randomly generated profile
P <- construct_profile(num_voters,C)
#winner <- f(profile = P,voting_rule = "Borda")


## Deciding an Election in this Formalism

One of the upshots of the formalism developed in Wayland2019 is that elections (as networks) are composable; a large election is simply a composition of smaller ones which can all be determined using the formalism above. My thesis specifically, shows that elections have a commutative monoidal structure. The monoidal structure allows for a formal understanding of not only composition, but parallel processing and agregating populations as well. For more detail see the full paper in the repo.


Now, lets take the presidential election as a prevelant example. Assuming we have the Democratic and Republican candidates already, the election process can be divided into two stages which we compose to determine the president elect. The two stages are:
* Deciding, by popular vote, the winning candidate of each state. This is done in parallel across the states.
* Doling out electoral college votes to candidates according to their standing in each state.

### State Vote
The first vote can be determined by plurality, where the winner of each state contribution is scored acoording to $\langle \text{1},0,...,0\rangle$. Simply, the candidate with the most votes is determined to be the winner of the state.

### Electoral College
This stage is a bit more subtle, both in this formalism and it's political conception. We know that in practice, the college's votes do not have to reflect the preferences of the people. For the purpose of this project, we will, in good faith, assume they do. In the spirit of composition, treat the states as a type of voter and use the state's "ballot" to assign electoral votes to decide the winner of the election. This is assuming all involved maintain a baseline level of integrity. Again we take this as a given, but 2020 has showed us this might not always be the case. The fact that we need to distinguish between states makes the construction of the scoring vector a bit more interesting. Notice that this formalism does not catalogue much information about specific voters aside from the cardinality and a simple ordering. In many cases it is only necessary to treat your set of voters as the set $\{1,...,n\}$.

#### A brief aside
* Although this may seem unproductive at first, this abstraction allows voting systems to be treated as a network, and thus leverage results of network theory. Understanding the underlying structure (category theory) allows a single formalism to capture the beahvior of elections, voting methods, feynman diagrams, and many other manifestations of a "network". One of the goals of this project is to start developoing an understanding of which parts of the formalism are important in practice.

In accordance with the construction in Wayland2019, the information about specific voters (states with differing electoral influence) must be encoded into the ballot and the scoring vector. This allows a consistent shift from the input to the network (the voters ballots) to the output (a designated winner). In this case output from the state vote becomes the ballot of the electoral college state. This leads to the following construction:


#### Step 1:
Given that this formalism is currently restricted to positional scoring rules, the most natural scoring vector to use is the list of electoral votes from largest to smallest. I.e. 
$$ S =  \langle 55,38,....,3\rangle$$
Where 55 corresponds to California's electoral votes, 38 to Texas's, et cetera. Essentially we are constructing the scoring vector to reflect an order of states according to population. Notice $|S|=51$.

  
#### Step 2:
We need to generate a single ballot that encodes the choice of each state. Recall that, formally, the ballot is simply a linear order on the set of candidates. For the scope of this project I won't go into details of how to construct the encoding, but rather just give the result: the generated ballot in its final form will tag the winning candidate and the reference state as such:

$$  B = \langle\{Biden,California\},\{Trump,Texas\},...\} \rangle $$

Again, as expected, we have $|B|=51$. The subtlety here comes from the fact that we need this to represent a linear order; it does, but getting there formally takes a bit of work to design the appopriate, isomorphic set and order.

#### Step 3:
Now we can match up the elements in B and S using a total function. Looking at the first element as example, the total function would assign 55 votes to Biden as California's contribution. Then proceed through the 50 remaining states to produce the total votes for Trump and Biden, and take the maximum to determine the winner. Combining this information into a single ballot keeps with the tradition of keeping voters indistinguishable that is baked into my formalism.
    
    
## Deciding in Practice 
This project looks at the  2016 election as an example since all the data is easily accessible. Pulling data from https://electionlab.mit.edu/ using https://github.com/MEDSL/elections.git :

In [43]:
#Loading data from 2016 Presidential election
data(presidential_precincts_2016)
data <- presidential_precincts_2016 %>%
  select(state, candidate, office, votes)
#Populate the table of results
dt <- data.table(data)
head(data)

state,candidate,office,votes
<chr>,<chr>,<chr>,<dbl>
Alabama,Hillary Clinton,US President,135
Alabama,Gary Johnson,US President,0
Alabama,Jill Stein,US President,1
Alabama,Donald Trump,US President,218
Alabama,[Write-in],US President,4
Alabama,Alabama Democratic Party,Straight Party,119


Note, the original query gives much more information than necessary. All that will be needed for this exercise is the candidates running for president and their performance in each state.

### Constructing Scoring Vector
Read in the electoral votes for each state. CSV file is provided in the repository.

In [36]:
#Get scoring vector for each state
electoral_votes <- read.csv(file = 'electoral_votes_per_state.csv')[,1:2]
electoral_votes$winner <- replicate(51,"Undecided")
colnames(electoral_votes) <- c("State","Electoral_Votes","Winner")
head(electoral_votes)

,State,Electoral_Votes,Winner
,<chr>,<int>,<chr>
1,Alabama,9,Undecided
2,Alaska,3,Undecided
3,Arizona,11,Undecided
4,Arkansas,6,Undecided
5,California,55,Undecided
6,Colorado,9,Undecided


In practice is is easier to deal with ordering the electoral votes alphabetically, which is a valid ordering.

In [37]:
#Order states in data table alphabetically to match scoring vector
states <- unique(data[order(state)]$state)

In [38]:
# Generate list of Candidates
candidates <- unique(data[order(candidate)]$candidate)

### Step 1:
Count the total votes for each candidate in each state using plurality scoring rule. This emulates the behavior of the total function outlined above, in practice.

In [46]:
count <- dt[,list(total_votes = sum(votes)), by = c("candidate","state")]
count <- count[order(state)]
head(count)

candidate,state,total_votes
<chr>,<chr>,<dbl>
Hillary Clinton,Alabama,727869
Gary Johnson,Alabama,44373
Jill Stein,Alabama,9367
Donald Trump,Alabama,1317127
[Write-in],Alabama,20333
Alabama Democratic Party,Alabama,397566


Determine the state winner by choosing the candidate with the maximum number of votes. This emulates the behavior of the 𝑓 function. 

### Step 2:

When determining the state winner, the cell below generates the state's ballot. In practice this turns out to be a simplified version of $B$ as defined above, since the ordering is handled lexigraphically by R. Furthermore, this places the states single ballot it in a table to visualize how to easily combine with the scoring vector that was produced above.

In [50]:
for (i in 1:length(states)) {
    state <-  as.data.frame(count[count$state==states[i]])
    winner <- state[which.max(state[,3]),]$candidate
    electoral_votes$Winner[i] <- winner
}
head(electoral_votes)

,State,Electoral_Votes,Winner
,<chr>,<int>,<chr>
1,Alabama,9,Donald Trump
2,Alaska,3,Donald Trump
3,Arizona,11,Donald Trump
4,Arkansas,6,Donald Trump
5,California,55,Hillary Clinton
6,Colorado,9,Hillary Clinton


Note the analog to $B$ ballot is electoral_votes$Winner.
Now, we can easily combine the ballot and the scoring vector to determine the total for each candidate:

In [52]:
final_count <- as.data.table(electoral_votes)[,list(total_votes = sum(Electoral_Votes)), by = "Winner"]
final_count

Winner,total_votes
<chr>,<int>
Donald Trump,305
Hillary Clinton,233


## Conclusions
The meat of this project is showing that a natural computation of the 2016 election can be captured and encoded using my formalism in Wayland2019. Although there are some hoops to jump through, and currently only positional scoring rules are supported, it does reinforce that my constructions are robust. The coding in this project is not particularily interesting, but writing out these examples and verifying that natural computations are consistent has been a good test for my formalism. If there wasn't a consistent way to capture something as prevelant as the presidential election, then my formalism should have defintely been amended. This exercise defintely gave me more familiarity with how the overarching structure will project down into a particular regime.

Understanding the full scope of this research might increase your appretiation for this specific exercise. The idea is to develop a tree of abstraction, if you will. The top nodes are understanding how networks function on an incredibly abstract scale. This leverages category theory, and most of you reading this project will probably want to stay far away from the top. However, there are fundamental ideas and results that underpin how all networks behave and this is something that we as humans should understand, given their ubiquity in our technological world. That being said as you add more and more structure to a network to produce something familiar (i.e. move down the branches of the tree) very interesting qualities start appearing. For instance, in Social Choice theory, you get arrows theorem. Where in the specification of a network does this stem from? Does an a similar property occur in other regimes that can be attributed to a particular branch of specfication? Every field that uses networks is characterized by these specific types of results. It might be ambitiuous, but it seems this abstraction tree could be a worthwhile venture with the yield of understanding connections and themes across disciplines. So what I've done in this project is delve all the way down to the nuts and bolts of computation of elections, ensuring consitency with the high level categorical structure. The idea is to play around and experiment in a particular branch of the tree, from top to bottom, to gain intuition for the levels of the tree and how information propogates. I hope this metaphor helps.